In [38]:
import collections
import numpy as np
import os
import pandas as pd
import re
import tqdm

In [3]:
genre = "wb"

In [21]:
files = sorted(filter(lambda x: re.match(f"^train\\.preprocess_.*genre_{genre}.*baseline\\.tsv", x) is not None, os.listdir(os.path.join(os.getenv("DATA_DIR"), 
    "mica_text_coref/movie_coref/results/coreference/baselines/"))))
print(len(files))

5184


In [26]:
dfs = []
for file in tqdm.tqdm(files):
    df = pd.read_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/baselines", file), sep="\t")
    dfs.append(df)

100%|██████████| 5184/5184 [00:10<00:00, 507.20it/s]


In [27]:
df = pd.concat(dfs)

In [28]:
df.shape

(108864, 14)

In [29]:
df.columns

Index(['preprocess', 'genre', 'entity', 'merge_speakers',
       'provide_gold_mentions', 'remove_gold_singletons', 'split_len',
       'overlap_len', 'merge_strategy', 'metric', 'movie', 'P', 'R', 'F'],
      dtype='object')

In [45]:
df = df.rename(columns={"movie": "metric", "metric": "movie"})

In [46]:
df[~df.provide_gold_mentions & ~df.remove_gold_singletons & (df.movie == "all")].sort_values(by="F")

,preprocess,genre,entity,merge_speakers,provide_gold_mentions,remove_gold_singletons,split_len,overlap_len,merge_strategy,movie,metric,P,R,F
19,nocharacters,wb,speaker,False,False,False,2048,256,none,all,bcub,1.188510,54.874233,2.326628
20,nocharacters,wb,speaker,True,False,False,2048,512,max,all,ceafe,1.537168,36.038041,2.948567
20,nocharacters,wb,speaker,True,False,False,2048,512,average,all,ceafe,1.564838,36.686752,3.001643
20,nocharacters,wb,speaker,True,False,False,2048,512,before,all,ceafe,1.564838,36.686752,3.001643
19,nocharacters,wb,speaker,True,False,False,2048,256,none,all,bcub,1.580767,54.199134,3.071939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,addsays,wb,speaker,True,False,False,5120,128,before,all,muc,80.625386,88.885107,84.554011
18,addsays,wb,speaker,True,False,False,5120,128,average,all,muc,80.625386,88.885107,84.554011
18,addsays,wb,speaker,True,False,False,5120,256,average,all,muc,80.762106,88.751030,84.568315
18,addsays,wb,speaker,True,False,False,5120,256,before,all,muc,80.762106,88.751030,84.568315


In [47]:
print("preprocess", collections.Counter(df.preprocess))
print("entity", collections.Counter(df.entity))
print("merge_speakers", collections.Counter(df.merge_speakers))
print("provide_gold_mentions", collections.Counter(df.provide_gold_mentions))
print("remove_gold_singletons", collections.Counter(df.remove_gold_singletons))
print("split_len", collections.Counter(df.split_len))
print("overlap_len", collections.Counter(df.overlap_len))
print("merge_strategy", collections.Counter(df.merge_strategy))
print("metric", collections.Counter(df.metric))
print("movie", collections.Counter(df.movie))

preprocess Counter({'addsays': 36288, 'nocharacters': 36288, 'none': 36288})
entity Counter({'all': 36288, 'person': 36288, 'speaker': 36288})
merge_speakers Counter({False: 54432, True: 54432})
provide_gold_mentions Counter({False: 54432, True: 54432})
remove_gold_singletons Counter({False: 54432, True: 54432})
split_len Counter({2048: 27216, 3072: 27216, 4096: 27216, 5120: 27216})
overlap_len Counter({128: 36288, 256: 36288, 512: 36288})
merge_strategy Counter({'after': 18144, 'average': 18144, 'before': 18144, 'max': 18144, 'min': 18144, 'none': 18144})
metric Counter({'muc': 36288, 'bcub': 36288, 'ceafe': 36288})
movie Counter({'avengers_endgame': 15552, 'dead_poets_society': 15552, 'john_wick': 15552, 'prestige': 15552, 'quiet_place': 15552, 'zootopia': 15552, 'all': 15552})


In [48]:
df.to_csv("/proj/sbaruah/data/mica_text_coref/movie_coref/results/coreference/baselines/train.genre_wb.baseline.tsv", sep="\t", index=False)